# Covid-19 Dashboard

This dashboard presents up-to-date statistics on the Covid-19 pandemic in the form of interactive graphs.

It was made using data from [Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE), a government agency in the UK.

In [1]:
# import the required libraries
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import time

In [2]:
# embed matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in variables
with open("agevaccnorthwest.json", "rt") as INFILE:
    data_north_west=json.load(INFILE)

with open("agevaccnortheast.json", "rt") as INFILE:
    data_north_east=json.load(INFILE)

with open("agevaccsouthwest.json", "rt") as INFILE:
    data_south_west=json.load(INFILE)

with open("agevaccsoutheast.json", "rt") as INFILE:
    data_south_east=json.load(INFILE)

json_data = [data_north_west, data_north_east, data_south_west, data_south_east]

In [4]:
def min_age(agerange):
    """Extracts the lower limit of the age range so that the age bands can be sorted. Takes the age range
    as a parameter and returns the lower limit of the age range as an integer."""
    agerange = agerange.replace('+','') # remove the +
    start = agerange.split('_')[0]
    return int(start)

In [5]:
def assign_data(dataframe, region, datalist):
    """Assigns the data from the json file to the dataframe. Takes the dataframe, the region name and the data list as parameters."""
    for entry in datalist: 
        ageband=entry['age'] 
        if ageband != '50+' and ageband != '75+':
            dataframe.loc[ageband, region]=entry['cumVaccinationCompleteCoverageByVaccinationDatePercentage']

In [6]:
# wrangle the data into a dataframe
def wrangle_data(rawdata):
    """Wranles the data into a dataframe and assigns the data. Takes the raw data as a parameter and returns a filled-in dataframe."""
    # extract dictionary of the data from list
    datadic_north_west = data_north_west['data'][0] 
    datadic_north_east = data_north_east['data'][0] 
    datadic_south_west = data_south_west['data'][0] 
    datadic_south_east = data_south_east['data'][0] 

    # extract the data from the dictionary
    datalist_north_west = datadic_north_west['age_vacc']
    datalist_north_east = datadic_north_east['age_vacc']
    datalist_south_west = datadic_south_west['age_vacc']
    datalist_south_east = datadic_south_east['age_vacc']

    # extract the age bands from the data (only need to do this on one data set, as the age bands are all the same)
    ageranges=[x['age'] for x in datalist_north_west]

    # use min_age function to sort the age bands 
    ageranges.sort(key=min_age)
    #remove redundant age bands
    ageranges.remove('50+') # remove the 50+ age band as it is does not contain any data
    ageranges.remove('75+') # remove the 75+ age band as it is does not contain any data

    # create a dataframe with the age bands as the index and the regions as the columns
    age_vacc = pd.DataFrame(index=ageranges, columns=['North West', 'North East', 'South West', 'South East'])

    # call the function to assign the data to the dataframe
    assign_data(age_vacc, 'North West', datalist_north_west)
    assign_data(age_vacc, 'North East', datalist_north_east)
    assign_data(age_vacc, 'South West', datalist_south_west)
    assign_data(age_vacc, 'South East', datalist_south_east)

    return age_vacc

age_vacc = wrangle_data(json_data)

## Download Current Data

Pressing the Refresh button below will ensure that the graphs reflect the current data available from PHE.

In [7]:
def access_api(): 
    """ Accesses the PHE API and returns current raw data."""
    # filter the data received in response to a request
    filters_north_west = [
    'areaType=region',
    'areaName=North West'
    ]   

    filters_north_east = [
    'areaType=region',
    'areaName=North East'
    ]  

    filters_south_west = [
    'areaType=region',
    'areaName=South West'
    ]   

    filters_south_east = [
    'areaType=region',
    'areaName=South East'
    ]  
    
    # define the metrics and structure received in response to a request
    structure = {
    "age_vacc": "vaccinationsAgeDemographics"
    }

    # create an instance of the API by passing the filters and structure to the constructor
    api_north_west = Cov19API(filters=filters_north_west, structure=structure)
    api_north_east = Cov19API(filters=filters_north_east, structure=structure)
    api_south_west = Cov19API(filters=filters_south_west, structure=structure)
    api_south_east = Cov19API(filters=filters_south_east, structure=structure)

    # send the request to the server and get the response in JSON format
    age_vacc_north_west = api_north_west.get_json()
    time.sleep(1) # wait 1 second to avoid overloading the API
    age_vacc_north_east = api_north_east.get_json()
    time.sleep(1)
    age_vacc_south_west = api_south_west.get_json()
    time.sleep(1)
    age_vacc_south_east = api_south_east.get_json()
    
    return [age_vacc_north_west, age_vacc_north_east, age_vacc_south_west, age_vacc_south_east]

In [8]:
def api_button_callback(button):
    """ Button callback - Accesses API, wrangles data, updates global variable df used for plotting. """
    # get current data from API
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global age_vacc
    age_vacc = wrangle_data(apidata)
    # simulate interaction with graph to refresh it
    refresh_graph()
    # update button icon to indicate data has been updated and disable it
    apibutton.icon="check"
    apibutton.disabled=True

# create a button to access the API
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download'
)

# call the function when the button is clicked
apibutton.on_click(api_button_callback) 

display(apibutton)

Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Graphs

This graph shows the pecentage of people who have received a Covid-19 vaccination since the start of the pandemic, divided by age group. The age groups are shown along the x-axis, with the percentage on the y-axis. The graph has been divided into regions, North West, North East, South West and South East, which you can select from the dropdown menu.

Total percentage of people who have received a COVID-19 vaccination since the start of the pandemic, by age group

In [9]:
def plot_age_vacc(region):
    """Plots the vaccination coverage by age band for the region selected. Takes the region name as a parameter."""
    age_vacc[region].plot(kind='bar', y=list(region))
    plt.show()

# create a dropdown to select the region
regions=wdg.Dropdown(
    options=['North West', 'North East', 'South West', 'South East'],
    value='North West',
    description='Region: ',
    disabled=False,
)

def refresh_graph():
    """Refreshes the graph when the the data is refreshed."""
    current=regions.value
    if current==regions.options[0]:
        other=regions.options[1]
    else:
        other=regions.options[0]
    regions.value=other # this forces the redraw
    regions.value=current # this changes it back

# call the function when the dropdown is changed
graph=wdg.interactive_output(plot_age_vacc, {'region': regions})

display(regions, graph)

Dropdown(description='Region: ', options=('North West', 'North East', 'South West', 'South East'), value='Nort…

Output()

**Author and Copyright Notice** 2022 Created by Penelope Grayson, with code from Fabrizio Smeraldi (f.smeraldi@qmul.ac.uk - web), all rights reserved.